In [3]:
import pandas as pd
import json

In [4]:
file1_df = pd.read_json('raw_data/purchase_data.json')
file2_df = pd.read_json('raw_data/purchase_data2.json')
print (len(file1_df))
print (len(file2_df))

780
78


In [5]:
#Check to make sure columns are the same between both files
file1_df.columns,file2_df.columns

(Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'),
 Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'))

In [6]:
#append contents of file2 to file 1
pymoli_df = file1_df.append(file2_df)
#pymoli_df = pd.merge(file1_df,file2_df, on="", how="inner")

In [7]:
#player count
playerCount = len(pymoli_df['SN'])     #alternate way: playerCount = pymoli_df['SN'].count().mean()
print(playerCount)

858


In [8]:
#purchase analysis

#number of unique items
uniqueItemCount = len(pymoli_df['Item ID'].unique())

#average purchase price
averagePurchasePrice = pymoli_df["Price"].mean()

#total number of purchases
purchaseCount = pymoli_df['Price'].count() #pymoli_df.loc[pymoli_df['Price'] == 0.00] #search through purchases to make sure non are free/0.00

#total revenue
totalRevenue = pymoli_df['Price'].sum()
#pymoli_df['Price'].value_counts()
#summary_df = pd.DataFrame({"UniqueItems":uniqueItemCount})

#summary_df["test"] = averagePurchasePrice
price_df_summary = pd.DataFrame({'Unique Item Count':uniqueItemCount,'Average Purchase Price':averagePurchasePrice,'Purchase Count':purchaseCount,'Total Revenue':totalRevenue},index=[0])
price_df_summary.head()

,Average Purchase Price,Purchase Count,Total Revenue,Unique Item Count
0,2.930571,858,2514.43,184


In [9]:
#gender demographics
genderCount = pymoli_df['Gender'].value_counts()
percentageGender = pymoli_df['Gender'].value_counts()/len(pymoli_df)*100
genderDemo_summary_df = pd.DataFrame()
genderDemo_summary_df["Gender Count"] = genderCount
genderDemo_summary_df["Percentage Gender"] = percentageGender

In [10]:
genderDemo_summary_df.head()

,Gender Count,Percentage Gender
Male,697,81.235431
Female,149,17.365967
Other / Non-Disclosed,12,1.398601


In [19]:
#purchase analysis (gender-specific)
gender_df_price =  pymoli_df.groupby(['Gender']).mean()["Price"]
gender_df_count =  pymoli_df.groupby(['Gender']).count()["Price"]

#normalized totals = divide total purchase value by count of each gender in dataset
gender_df_normalized_total=gender_df_price/gender_df_count
gender_df_normalized_total_percentage = gender_df_normalized_total * 100

genderPurchase_summary_df = pd.DataFrame({'Average Price':gender_df_price,'Purchase Count':gender_df_count,'Normalized Totals':gender_df_normalized_total,'Normalized Totals %':gender_df_normalized_total_percentage})

In [20]:
genderPurchase_summary_df.head()

,Average Price,Normalized Totals,Normalized Totals %,Purchase Count
Gender,,,,
Female,2.847584,0.019111,1.911130,149
Male,2.944448,0.004224,0.422446,697
Other / Non-Disclosed,3.155000,0.262917,26.291667,12


In [17]:
#age demographics
bins = [1,9,14,19,24,29, 34, 39,44,49]
bin_names =["less than 10", "10-14", "15-19","20-24","25-29", "30-34","35-39", "40-44","45-49"]


In [24]:
age_df = pd.DataFrame(pd.cut(pymoli_df["Age"], bins, labels=bin_names).head())
age_df.head()

,Age
0,35-39
1,20-24
2,30-34
3,20-24
4,20-24
